<a href="https://colab.research.google.com/github/BIueMan/ImageSelfClustering/blob/main/Graph_SSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import cv2
import time
from google.colab.patches import cv2_imshow
import os

In [ ]:
# Connect to drive to save work
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/EE049064

im = cv2.cvtColor(cv2.imread('./amanita.jpg'), cv2.COLOR_BGR2RGB)

# Auxilary

In [ ]:
def toy_image(imgSz, refSz=None):
  return np.random.random(*imgSz) , np.random.random(*imgSz), np.random.random(*imgSz) 

In [ ]:
def nullFunc():
  return None

In [ ]:
def save_img(img, imgFn):
  return None

def save_mtx(smp, mtxFn):
  return None


In [ ]:
def load_img(imgFn):
  return None

def load_mtx(mtxFn):
  return None


In [ ]:
def show_image(image, grid=None):
  return None


# Configuration class

In [ ]:
def defaultDict():
  dct = {'name': 'imageDir', 
         'patchSize': 9,
         'patchStep': 9}
  return dct

In [ ]:
class configuration:
  def __init__(self, dct=defaultDict()):
    self.dct = dct

    dn = os.path.join(os.getcwd(), dct['name'])
    patchSize = dct['patchSize']
    patchStep = dct['patchStep']


    self.imgFn = os.path.join(dn, 'image.jpg')
    self.smpFn = os.path.join(dn, 'sample')
    self.refFn = os.path.join(dn, 'ref')
    self.ptcFn = os.path.join(dn, 'patches', 'pt')
    self.wgtFn = os.path.join(dn, 'weights')
    self.resFn = os.path.join(dn, 'result')

    self.splitFunc = dct.get("splitFunc", splitPatches(patchSize, patchStep))
    self.preprocFunc = dct.get("preprocFunc", colorConvert())
    self.weightFunc = dct.get("weightFunc", nullFunc)
    self.solveFunc = dct.get("solveFunc", nullFunc)
    self.compareFunc = dct.get("compareFunc", nullFunc)

# Preprocess functions

In [ ]:
class splitPatches:
  def __init__(self, sz, st):
    self.sz = sz
    self.st = st
  
  #def __call__(self, img)


class colorConvert:
  def __init__(self, scheme="gray"):
    self.scheme = scheme

  #def __call__(self, patches)

## Auto encoder

# Weight functions

# Solve functions

# Compare functions

# Image class

In [ ]:
class Image:
  def __init__(self, img, smp, ref, config=configuration()):
    self.image = img
    self.sample = smp
    self.ref = ref

    self.config = config

    self.patch = None
    self.weight = None
    self.result = None
    self.error = None

    self.load(config)

    if self.image is None:
      print("No input image given")

  def Preprocess(self):
    split = self.splitFunc(self.image)
    self.patch = self.preprocFunc(split)

  def CalcWeights(self):
    if self.patches is None:
      print("Patches don't exist")
    else:
      self.weight = self.weightFunc(self.patch)

  def Solve(self):
    if self.weight is None:
      print("Weights don't exist")
    else:
      self.result = self.solveFunc(self.weight)

  def Compare(self):
    if self.result is None:
      print("Results don't exist")
    else:
      if self.ref is None:
        print("There is no reference")
      else:
        self.error = self.compareFunc(self.result, self.ref)
        
  def __call__(self):
    self.Preprocess()
    self.CalcWeights()
    self.Solve()
    self.Compare()
    self.save()
  
  def load(self, config):
    self.image = load_img(self.config.imgFn)
    self.sample = load_mtx(self.config.smpFn)
    self.ref = load_mtx(self.config.refFn)
    self.patch = load_img(self.config.ptcFn)
    self.weight = load_mtx(self.config.wgtFn)
    self.result = load_mtx(self.config.resFn)

    self.splitFunc = self.config.splitFunc
    self.preprocFunc = self.config.preprocFunc
    self.weightFunc = self.config.weightFunc
    self.solveFunc = self.config.solveFunc
    self.compareFunc = self.config.compareFunc

  def save(self):
    save_img(self.image, self.config.imgFn)
    save_mtx(self.sample, self.config.smpFn)
    save_mtx(self.ref, self.config.refFn)
    save_img(self.patch, self.config.ptcFn)
    save_mtx(self.weight, self.config.wgtFn)
    save_mtx(self.result, self.config.resFn)
    save_mtx(self.error, self.config.errFn)

  def show(self):
    plt.figure(figsize=(15, 45))

    plt.subplot(321)
    show_image(self.image)
    plt.title("Image")
    plt.axis('off')

    plt.subplot(322)
    show_image(self.image, self.sample)
    plt.title("Sample data")
    plt.axis('off')

    plt.subplot(323)
    show_image(self.image, self.result)
    plt.title("Results")
    plt.axis('off')

    plt.subplot(324)
    show_image(self.image, self.ref)
    plt.title("Ref")
    plt.axis('off')

    plt.subplot(313)
    show_image(self.error)
    plt.title("Error")
    plt.axis('off')

